In [1]:
import numpy
# fix random seed for reproducibility
numpy.random.seed(7)
import pandas as pd
import numpy as np
from __future__ import division, print_function, unicode_literals


import matplotlib
import matplotlib.pyplot as plt

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

tf.reset_default_graph()
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

np.random.seed(42)
tf.set_random_seed(42)


from keras.models import Sequential
from keras.layers import Dense
import numpy
# fix random seed for reproducibility
numpy.random.seed(7)
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import confusion_matrix,precision_recall_curve,auc,roc_auc_score,roc_curve,recall_score,classification_report,accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from sklearn.preprocessing import StandardScaler
from keras import regularizers

/home/inderjeet78/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data = pd.read_csv("creditcard.csv")
data['normAmount'] = StandardScaler().fit_transform(data['Amount'].values.reshape(-1, 1))
data = data.drop(['Time','Amount'],axis=1)
X = data.iloc[:,data.columns != 'Class']
Y = data.iloc[:,data.columns == 'Class']
pca = PCA(n_components=25)
X = pca.fit_transform(X)
X_train,testX,Y_train,testY = train_test_split(X,Y,test_size=0.20,random_state=21, stratify=Y)
X_train,valX,Y_train,valY = train_test_split(X_train,Y_train,test_size=0.20,random_state=21, stratify=Y_train)

In [3]:
X_train = np.array(X_train)
valX = np.array(valX)
testX = np.array(testX)
Y_train = np.array(Y_train)
valY = np.array(valY)
testY = np.array(testY)

# Model Prepration

In [4]:
# create model
model = Sequential()
model.add(Dense(32, input_dim=25, activation='relu'))
model.add(Dense(48, activation='relu'))
model.add(Dense(32, activation='tanh'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
model.fit(X_train, Y_train, epochs=10, batch_size=256)

# evaluate the model
scores = model.evaluate(valX, valY)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# calculate predictions
predictions = model.predict(testX)
predictionst = model.predict(X_train)
predictionsval = model.predict(valX)
# round predictions
rounded = [round(x[0]) for x in predictions]
rounded2 = [round(x[0]) for x in predictionst]
roundedval = [round(x[0]) for x in predictionsval]

Epoch 1/10
182276/182276 [==============================] - 2s 13us/step - loss: 0.0302 - acc: 0.9929
Epoch 2/10
182276/182276 [==============================] - 2s 9us/step - loss: 0.0032 - acc: 0.9994
Epoch 3/10
182276/182276 [==============================] - 2s 9us/step - loss: 0.0029 - acc: 0.9994
Epoch 4/10
182276/182276 [==============================] - 2s 9us/step - loss: 0.0027 - acc: 0.9994
Epoch 5/10
182276/182276 [==============================] - 2s 10us/step - loss: 0.0026 - acc: 0.9994
Epoch 6/10
182276/182276 [==============================] - 2s 10us/step - loss: 0.0025 - acc: 0.9994
Epoch 7/10
182276/182276 [==============================] - 2s 9us/step - loss: 0.0024 - acc: 0.9994
Epoch 8/10
182276/182276 [==============================] - 2s 9us/step - loss: 0.0023 - acc: 0.9994
Epoch 9/10
182276/182276 [==============================] - 2s 8us/step - loss: 0.0022 - acc: 0.9995
Epoch 10/10
45569/45569 [==============================] - 1s 23us/step

acc: 99.94%


# Training Set Predictions

In [5]:
print("Train roc auc score is : ", roc_auc_score(Y_train,rounded2))
print("Accuracy : ",accuracy_score(Y_train,rounded2))
print("Classification report is : ")
print(classification_report(Y_train, rounded2))
print("Confusion Matrix for test set is : ")
print(confusion_matrix(Y_train,rounded2))

Train roc auc score is :  0.9268577262718865
Accuracy :  0.9994952709078541
Classification report is : 
             precision    recall  f1-score   support

          0       1.00      1.00      1.00    181961
          1       0.85      0.85      0.85       315

avg / total       1.00      1.00      1.00    182276

Confusion Matrix for test set is : 
[[181915     46]
 [    46    269]]


# Validation set predictions

In [6]:
print("Test roc auc score is : ", roc_auc_score(valY,roundedval))
print("Accuracy : ",accuracy_score(valY,roundedval))
print("Classification report is : ")
print(classification_report(valY,roundedval))
print("Confusion Matrix for test set is : ")
print(confusion_matrix(valY,roundedval))

Test roc auc score is :  0.8796698954562276
Accuracy :  0.9994294366784436
Classification report is : 
             precision    recall  f1-score   support

          0       1.00      1.00      1.00     45490
          1       0.90      0.76      0.82        79

avg / total       1.00      1.00      1.00     45569

Confusion Matrix for test set is : 
[[45483     7]
 [   19    60]]


# Test Set Predictions

In [7]:
print("Test roc auc score is : ", roc_auc_score(testY,rounded))
print("Accuracy : ",accuracy_score(testY,rounded))
print("Classification report is : ")
print(classification_report(testY, rounded))
print("Confusion Matrix for test set is : ")
print(confusion_matrix(testY,rounded))

Test roc auc score is :  0.923346287023532
Accuracy :  0.9994908886626171
Classification report is : 
             precision    recall  f1-score   support

          0       1.00      1.00      1.00     56864
          1       0.86      0.85      0.85        98

avg / total       1.00      1.00      1.00     56962

Confusion Matrix for test set is : 
[[56850    14]
 [   15    83]]
